<a href="https://colab.research.google.com/github/jwattspajaro/CS50-s/blob/main/PDF_registr_csv_limpio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-

import pdfplumber
import csv
import re
import uuid#!/usr/bin/env python3

import os
import sys
from collections import OrderedDict
import io

# ---------------- CONFIG ----------------
PDF_FILE = "20231006_puestos-de-votacion-georreferenciacion_territoriales.pdf"
OUTPUT_CSV = "puestos_votacion_limpio.csv"
LOCAL_DANE_CSV = "DIVIPOLA-_C_digos_municipios.csv"
# ----------------------------------------

# Reemplazos en nombres de puestos
REPLACEMENTS_PUESTO = [
    (r'\bI\.?E\.?\b', 'Institución Educativa'),
    (r'\bIE\b', 'Institución Educativa'),
    (r'\bInst\.?Educ\.?\b', 'Institución Educativa'),
    (r'\bInst Educ\b', 'Institución Educativa'),
    (r'\bInstit\.\b', 'Institución'),
    (r'\s{2,}', ' ')
]

# Reemplazos en direcciones
REPLACEMENTS_DIRECCION = [
    (r'\bCra\b\.?', 'Carrera'),
    (r'\bCR\b\.?', 'Carrera'),
    (r'\bCr\b\.?', 'Carrera'),
    (r'\bCRA\b\.?', 'Carrera'),
    (r'\bCl\b\.?', 'Calle'),
    (r'\bCll\b\.?', 'Calle'),
    (r'\bCLL\b\.?', 'Calle'),
    (r'\bAv\b\.?', 'Avenida'),
    (r'\bAvda\b\.?', 'Avenida'),
    (r'\bTransv\b\.?', 'Transversal'),
    (r'\bTrans\b\.?', 'Transversal'),
    (r'\s{2,}', ' ')
]

def capitalizar(texto: str) -> str:
    texto = texto.strip()
    if not texto:
        return ""
    return " ".join([w.capitalize() for w in re.split(r'\s+', texto.lower())])

def apply_replacements(text: str, repl_list):
    s = text or ""
    for pattern, repl in repl_list:
        s = re.sub(pattern, repl, s, flags=re.IGNORECASE)
    return re.sub(r'\s{2,}', ' ', s).strip()

def cargar_dane_lookup():
    dept_codes = {}
    mun_codes = {}

    if os.path.exists(LOCAL_DANE_CSV):
        print("Usando tabla DANE local:", LOCAL_DANE_CSV)
        with open(LOCAL_DANE_CSV, 'r', encoding='utf-8') as f:
            csv_text = f.read()
    else:
        print("ERROR: No se encontró el archivo de códigos DANE:", LOCAL_DANE_CSV)
        return dept_codes, mun_codes

    reader = csv.DictReader(io.StringIO(csv_text))
    for row in reader:
        dep = row.get("Nombre Departamento") or row.get("nombre departamento")
        dep_code = row.get("Código Departamento") or row.get("codigo departamento")
        mun = row.get("Nombre Municipio") or row.get("nombre municipio")
        mun_code = row.get("Código Municipio") or row.get("codigo municipio")

        if dep and dep_code:
            dept_codes[dep.upper()] = dep_code
        if dep and mun and mun_code:
            mun_codes[(dep.upper(), mun.upper())] = mun_code

    return dept_codes, mun_codes

def parse_pdf_to_records(pdf_path):
    records = []

    if not os.path.exists(pdf_path):
        print("ERROR: no se encontró el PDF:", pdf_path)
        sys.exit(1)

    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            table = page.extract_table()
            if table and len(table) > 1:
                for row in table[1:]:
                    row = [(c or "").strip() for c in row]
                    lat, lon = "", ""
                    for i in range(len(row)-1, -1, -1):
                        if re.search(r'[-]?\d[\d\.,]+', row[i]):
                            if not lon:
                                lon = row[i]
                            elif not lat:
                                lat = row[i]
                                break
                    dept = row[0] if len(row) > 0 else ""
                    mun = row[1] if len(row) > 1 else ""
                    puesto = row[2] if len(row) > 2 else ""
                    direccion = row[3] if len(row) > 3 else ""

                    lat = lat.replace(',', '.').strip()
                    lon = lon.replace(',', '.').strip()

                    reg = {
                        "departamento_raw": dept,
                        "municipio_raw": mun,
                        "puesto_raw": puesto,
                        "direccion_raw": direccion,
                        "latitud_raw": lat,
                        "longitud_raw": lon
                    }
                    records.append(reg)

    return records

def normalize_and_enrich(records, dept_lookup, mun_lookup):
    out = []
    seen = set()

    for r in records:
        dep_raw = (r.get("departamento_raw") or "").strip()
        mun_raw = (r.get("municipio_raw") or "").strip()
        puesto_raw = (r.get("puesto_raw") or "").strip()
        direccion_raw = (r.get("direccion_raw") or "").strip()
        lat_raw = (r.get("latitud_raw") or "").strip()
        lon_raw = (r.get("longitud_raw") or "").strip()

        puesto_norm = capitalizar(apply_replacements(puesto_raw, REPLACEMENTS_PUESTO))
        direccion_norm = capitalizar(apply_replacements(direccion_raw, REPLACEMENTS_DIRECCION))
        departamento_norm = capitalizar(dep_raw)
        municipio_norm = capitalizar(mun_raw)

        cod_dep = dept_lookup.get(departamento_norm.upper(), "")
        cod_mun = mun_lookup.get((departamento_norm.upper(), municipio_norm.upper()), "")

        lat, lon = "", ""
        try:
            latf = float(lat_raw.replace(',', '.')) if lat_raw else ""
            lonf = float(lon_raw.replace(',', '.')) if lon_raw else ""
            lat = "{:.8f}".format(latf) if latf != "" else ""
            lon = "{:.8f}".format(lonf) if lonf != "" else ""
        except:
            pass

        maps_url = f"https://www.google.com/maps?q={lat},{lon}" if lat and lon else ""

        key = (departamento_norm.upper(), municipio_norm.upper(), puesto_norm.upper(), direccion_norm.upper(), lat, lon)
        if key in seen:
            continue
        seen.add(key)

        rec = OrderedDict()
        rec["id"] = str(uuid.uuid4())
        rec["departamento"] = departamento_norm
        rec["codigo_dane_departamento"] = cod_dep
        rec["municipio"] = municipio_norm
        rec["codigo_dane_municipio"] = cod_mun
        rec["puesto_votacion"] = puesto_norm
        rec["direccion"] = direccion_norm
        rec["latitud"] = lat
        rec["longitud"] = lon
        rec["google_maps_url"] = maps_url

        out.append(rec)

    return out

def save_csv(records, out_file):
    if not records:
        print("No hay registros para guardar.")
        return
    with open(out_file, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=list(records[0].keys()))
        writer.writeheader()
        for r in records:
            writer.writerow(r)
    print(f"CSV guardado: {out_file} ({len(records)} registros)")

def main():
    print("1) Cargando tabla DANE...")
    dept_lookup, mun_lookup = cargar_dane_lookup()
    print(f" - Deptos: {len(dept_lookup)}, Municipios: {len(mun_lookup)}")

    print("2) Extrayendo datos del PDF...")
    raw_records = parse_pdf_to_records(PDF_FILE)
    print(f" - Registros crudos: {len(raw_records)}")

    print("3) Normalizando y enriqueciendo...")
    final = normalize_and_enrich(raw_records, dept_lookup, mun_lookup)
    print(f" - Registros finales: {len(final)}")

    print("4) Guardando CSV...")
    save_csv(final, OUTPUT_CSV)
    print("¡Listo!")

if __name__ == "__main__":
    main()